# Physics 494/594
## Batch Processing and Linear Algebra with Numpy

<img src='https://numpy.org/images/logo.svg' width=300px>

In [ ]:
# %load ./include/header.py
import numpy as np
import matplotlib.pyplot as plt
import sys
from tqdm import trange,tqdm
sys.path.append('./include')
import ml4s
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.style.use('./include/notebook.mplstyle')
np.set_printoptions(linewidth=120)
ml4s.set_css_style('./include/bootstrap.css')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

## Last Time

### [Notebook Link:04_Visualizing_NN_Output.ipynb](./04_Visualizing_NN_Output.ipynb)

- Understand the output of a neural network with 2 input neurons via visualization

## Today
- Explore linear algebra in `numpy` for batch processing of samples
- More information about doing linear algebra in `numpy` is included in [tutorial_numpy_arrays.ipynb](./tutorial_numpy_arrays.ipynb)

### `numpy` is **much** faster than built in python lists

Here I'm using an iPython cell `magic` to time code in a given cell.  You can see a list of all magics [here](https://ipython.readthedocs.io/en/stable/interactive/magics.html).

Let's create a list of all integers up to $10^{7}$.

In [ ]:
%%time
num_elements = 10_000_000
x1 = []
for i in range(num_elements):
    x1.append(i*i)
    
print(x1[-10:])

In [ ]:
%%time
x2 = np.arange(num_elements)**2
print(x1[-10:])

#### Matrix Multiplication
`np.dot` contracts over the innermost index. We need to think carefully about this when considering batches of training (input) samples.  We have also seen that we can use the `@` symbol for matrix multiplication.

In [ ]:
N = [8,7]
Nsamples = 30

W = np.random.uniform(low=-10,high=10,size=[N[1],N[0]])
x = np.zeros([N[0],Nsamples])

# Now perform the dot product (b = 0 for now)
z = np.dot(W,x)
print(z.shape)

<div class="span alert alert-danger">
This is a problem! $z$ has dimensions $N_1 \times N_{\rm samples}$. Let's see what happens when we try to add the biases.
    </div>

In [ ]:
b = np.random.uniform(low=-1,high=1,size=[N[1]])
z = np.dot(W,x) + b

We can fix this with the re-ordering if indices we discussed:

In [ ]:
# But with a re-ordering of indices, this works!
# So, let's take the dimension of size 30 to be
# the very first one:
W = np.random.uniform(low=-10,high=10,size=[N[0],N[1]])
x = np.zeros([Nsamples,N[0]])

print(np.dot(x,W).shape)

Now we can add the bias vector and we will get the correct output shape.

In [ ]:
z = np.dot(x,W)+b
print(z.shape)

### Modified Apply Network Functions
We need to make a small modification to our `feed_forward` function to reflect the new ordering.

`np.dot(w[ℓ],a)` $\to$ `np.dot(a,w[ℓ])`

In [ ]:
def feed_forward(a0,w,b):
    num_layers = len(w)
    a = a0
    for ℓ in range(num_layers):
        z = np.dot(a,w[ℓ]) + b[ℓ]
        a = 1.0/(1.0+np.exp(-z))
    return a

Note that we have changed the code: `N[ℓ],N[ℓ-1]` to `N[ℓ-1],N[ℓ]` as $w$ is now a $N_{\ell-1} \times N_{\ell}$ matrix!

In [ ]:
N = [9,5,1]
w,b = [],[]

# Note: we have updated the matrix dimensions and now start the loop at 1
for ℓ in range(1,len(N)):
    w.append(np.random.uniform(low=-10,high=10,size=(N[ℓ-1],N[ℓ])))
    b.append(np.random.uniform(low=-1,high=1, size=N[ℓ]))

Apply the network `batch_size` times in parallel!

In [ ]:
%%time
batch_size=10000
x = np.random.uniform(low=-1,high=1,size=(batch_size,N[0]))
a_out = feed_forward(x,w,b)

Check the shape of the output

In [ ]:
a_out.shape

## Efficient Visualization of our Simple Neural Network

In [ ]:
N = [2,400,400,1]
w,b = [],[]
for ℓ in range(len(N)-1):
    w.append(np.random.uniform(low=-10,high=10,size=(N[ℓ],N[ℓ+1])))
    b.append(np.random.uniform(low=-1,high=1, size=N[ℓ+1]))

### Using Meshgrid to Evaluate in Parallel

In [ ]:
grid_size = 200 # the size of the grid of input values

# the box size
x0_min,x0_max = -1.0,1.0
x1_min,x1_max = -1.0,1.0

x = np.meshgrid(np.linspace(x0_min,x0_max,grid_size),np.linspace(x1_min,x1_max,grid_size))

In [ ]:
fig,ax = plt.subplots(1,2, figsize=(8,4))
for i,cax in enumerate(ax):
    pcm = cax.pcolormesh(x[i],cmap='Spectral_r', rasterized=True)
    cax.set_title(f'$x_{i}$')
    cax.set_aspect('equal')

fig.tight_layout()
fig.colorbar(pcm, ax=ax[:], shrink=0.6, label='Coordinate Value');

Now we need to flatten as our network always takes a 1D array as input

In [ ]:
xflat = [x[0].flatten(),x[1].flatten()]
np.shape(xflat[0]) # this should be grid_size^2

In [ ]:
batch_size = grid_size**2
a0 = np.zeros([batch_size,2])

a0[:,0]=xflat[0] # fill first component (index 0)
a0[:,1]=xflat[1] # fill second component

### Now we can apply the network for each sample in parallel

In [ ]:
%%time
a1 = feed_forward(a0,w,b)

### Plot the result

In [ ]:
a1 = a1.reshape(grid_size,grid_size) # reshape for plotting

plt.imshow(a1, extent=[-1,1,-1,1], cmap='Spectral_r', rasterized=True, interpolation='lanczos', origin='lower')
plt.colorbar(label='Activation')
plt.xlabel(r'$x_1$')
plt.ylabel(r'$x_0$');

### Can also visualize in 3D!

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(8,6))
ax = plt.axes(projection='3d')

surf = ax.plot_surface(x[0], x[1],a1 , rstride=1, cstride=1, cmap='Spectral_r', 
                       linewidth=0, antialiased=True, rasterized=True)
ax.set_xlabel(r'$x_0$',labelpad=8)
ax.set_ylabel(r'$x_1$',labelpad=8)
ax.set_zlabel(r'$f(\mathbf{x} )$',labelpad=8);